# **빅데이터 개론 Lab 10 - 로지스틱 회귀분석**

참고자료 : https://www.notion.so/TA-2689a38b5289413a82671d3956fea103

- - -




### **<로지스틱 회귀분석>**

<img src="https://mblogthumb-phinf.pstatic.net/MjAyMDAzMTNfMjUg/MDAxNTg0MDgwNjAzNDQ4.stg44EjmKsUiCbav6P4HPZpzQocCNcN3L6DUwQBC5lgg.25bRo94dsEGXtmZ-Z0CtQhXKlTVkkz3E5VPaq0oNvXog.PNG.y4769/%EC%8A%AC%EB%9D%BC%EC%9D%B4%EB%93%9C4.PNG?type=w800" width="600" height="300">


<br>

### **<독립변수(설명변수)와 종속변수(반응변수)의 관계>**

<img src="https://mblogthumb-phinf.pstatic.net/MjAyMDAzMTNfMTc3/MDAxNTg0MDgwNjAzNDQy.lXx6Ca-LIa0kaeCULOE9wWqUQsZt1KzSOZDKsRpwbREg._lAOicl-kDpLz4SYa9_37_rYCtUNhL_IYFOMYz4v-Hgg.PNG.y4769/%EC%8A%AC%EB%9D%BC%EC%9D%B4%EB%93%9C6.PNG?type=w800" width="600" height="300">

  1. 선형 회귀분석이 말 그대로 독립변수와 종속변수 사이의 선형적 관계를 그래프로 나타낸 것이라면, 로지스틱 회귀분석은 선형이 아닌 "S" 곡선의 특성을 나타낸다.

  2. 로지스틱 회귀분석의 곡선에서 종속변수의 확률이 0.5보다 크면 어떤 사건이 일어난다고 보고, 0.5보다 작으면 어떤 사건이 일어나지 않을 것이라고 확률적으로 예측해볼 수 있다.




```
glm(formula, data, family = "binomial") 
# binomial 옵션은 반응변수가 두 가지의 값(이항)만 가진다고 glm에게 알려줌
```


In [ ]:
install.packages(c("tidyverse", "data.table", "caret", "e1071"))
library(tidyverse)
library(data.table)
options(repr.plot.width=10, repr.plot.height=10)

#### **A. 예제 1**

---


1. **HR(Human Resource, 인사관리)_연습데이터([링크](https://www.dropbox.com/sh/xx1w2syi768kfu0/AACZgxgo1fcxyDMgv9U-iTz8a?dl=0))**

* 변수 설명

```
satisfaction_level : 직무 만족도
last_evaluation : 마지막 평가점수
number_project : 진행 프로젝트 수
average_monthly_hours : 월평균 근무시간
time_spend_company : 근속년수
work_accident : 사건사고 여부(0: 없음, 1: 있음, 범주형)
left : 이직 여부(0: 잔류, 1: 이직, 범주형)
promotion_last_5years: 최근 5년간 승진여부(0: 승진 x, 1: 승진, 범주형)
sales : 부서(범주형)
salary : 임금 수준(범주형)
```


In [ ]:
hr_DF <- fread("https://raw.githubusercontent.com/Jin0331/TA/master/data/HR_comma_sep.csv", sep = ",") %>% as_tibble()

In [ ]:
hr_DF %>% str()

In [ ]:
hr_DF %>% summary()

* 범주형 변수 확인

In [ ]:
hr_DF$sales %>% unique() # 범주형~

In [ ]:
hr_DF$salary %>% unique()

In [ ]:
hr_DF$Work_accident %>% unique()

In [ ]:
hr_DF$left %>% unique()

In [ ]:
hr_DF$promotion_last_5years %>% unique()

* mutate를 이용한 데이터 타입 변경(int or chr ---> factor)

In [ ]:
hr_DF <- hr_DF %>% mutate_at(`.vars` = c("sales", "salary", "Work_accident", "left", "promotion_last_5years"), `.funs` = as.factor)
hr_DF %>% str()

* **train-test split**

In [ ]:
library(caret) 
set.seed(31)

index <- createDataPartition(y = hr_DF$left, p = 0.7, list = FALSE) 
head(index, 30)

In [ ]:
train <- hr_DF[index, ]
test <- hr_DF[-index, ]

train %>% show()
test %>% show()

* **train set을 이용한 glm 모델 생성**

In [ ]:
m <- glm(left ~ ., data = train, family = "binomial")
summary(m)

* feature selection

In [ ]:
m2 <- step(m, direction = "backward")

In [ ]:
summary(m2)

* sales 제거

In [ ]:
m3 <- update(m2, .~. -sales)
summary(m3)

* **예측**

In [ ]:
test %>% show()

In [ ]:
predict_value <- predict(m3, test, type = "response") %>% tibble(predict_value = .)
predict_value %>% show()

In [ ]:
predict_check <- test %>% select(left) %>% dplyr::bind_cols(., predict_value) 
predict_check %>% show()

* cut-off value 설정
 - predict_value > 0.5, left 1(이직)
 - predict_value <= 0.5, left 0(이직 하지 않음)

In [ ]:
predict_cutoff <- predict_check %>% mutate(predict_value_cutoff = as.factor(ifelse(predict_value > 0.5, 1, 0)))
predict_cutoff %>% show()

* Confusion Matrix(실제값과 모델에 의한 분류값을 비교하는 테이블)

![png](https://github.com/Jin0331/TA/blob/master/image/confusion_m.png?raw=true)

https://yamalab.tistory.com/50

In [ ]:
caret::confusionMatrix(predict_cutoff$left, predict_cutoff$predict_value_cutoff)

* ROC curve와 AUC
 - 여기서 하나 주의할 점은, 위에서 cut-off value를 0.5로 하여 분류했다는 것. 이 cut-off value가 중요한 이유는 그 값에 따라 분류가 천차만별이기 때문에, 모든 cut-off value를 고려하여 결과값의 변화를 살펴봐야하는데, 그 방법을 ROC curve라고 한다.

![png](https://github.com/Jin0331/TA/blob/master/image/AUC.png?raw=true)

In [ ]:
install.packages("pROC")

In [ ]:
library("pROC")

In [ ]:
roc_c <- roc(predict_cutoff$left, predict_cutoff$predict_value) # 실제정답과 glm 예측된 확률
plot.roc(roc_c,   
        col="royalblue",  
        print.auc=TRUE, 
        max.auc.polygon=TRUE,   
        print.thres=TRUE, print.thres.pch=19, print.thres.col = "red",
        auc.polygon=TRUE, auc.polygon.col="#D1F2EB")

In [ ]:
pROC::coords(roc_c, "best", ret = "threshold", transpose = F)

* sales를 제거하지 않은 m2 모델과 비교

In [ ]:
predict_value_m2 <- predict(m2, test, type = "response") %>% tibble(predict_value = .)
predict_check_m2 <- test %>% select(left) %>% dplyr::bind_cols(., predict_value_m2) 
roc_c_m2 <- roc(predict_check_m2$left, predict_check_m2$predict_value)

In [ ]:
plot.roc(roc_c,   
         col = "red",   
         print.auc=T, 
         max.auc.polygon=T,   
         print.thres=T, print.thres.pch=19, print.thres.col = "red", 
         auc.polygon=T, auc.polygon.col="#D1F2EB")  

plot.roc(roc_c_m2,   
         add=T,   # 기본 그래프에 추가할 수 있도록 설정
         col="blue",  
         print.auc=T, print.auc.adj=c(1.11,1.2),
         print.thres=T, print.thres.pch=19, print.thres.col = "blue")
         
legend("bottomright", legend=c("m3(-sales)", "m2"), col=c("red", "blue"), lwd = 2)  

- - -

#### **B. 예제 2**

* https://www.kaggle.com/c/titanic/data

**<kaggle의 타이타닉 data>**

  * survived : 생존=1, 죽음=0
  * pclass : 승객 등급. 1등급=1, 2등급=2, 3등급=3
  * sibsp : 함께 탑승한 형제 또는 배우자 수
  * parch : 함께 탑승한 부모 또는 자녀 수
  * ticket : 티켓 번호
  * cabin : 선실 번호
  * embarked : 탑승장소 S=Southhampton, C=Cherbourg, Q=Queenstown

In [ ]:
#@title
DF <- fread("https://raw.githubusercontent.com/Jin0331/TA/master/data/titanic/train.csv") %>%
 as_tibble()

In [ ]:
#@title
str(DF)

In [ ]:
#@title
DF %>% summary()

* 범주형 변수 확인

In [ ]:
#@title
DF$Survived %>% unique()

In [ ]:
#@title
DF$Pclass %>% unique()

In [ ]:
#@title
DF$Sex %>% unique()

In [ ]:
#@title
DF$Ticket %>% unique()

In [ ]:
#@title
DF$Embarked %>% unique()

In [ ]:
#@title
DF <- DF %>% 
 select(-PassengerId, -Name, -Cabin, -Ticket) %>% mutate_at(c("Survived","Sex","Embarked"), factor)
summary(DF)

* Hmisc::impute을 이용한 NA 값 대체(평균, 중앙값, 특정 숫자)

* https://m.blog.naver.com/PostView.nhn?blogId=tjdudwo93&logNo=221142961499&proxyReferer=https:%2F%2Fwww.google.com%2F

In [ ]:
#@title
library(Hmisc)
DF$Age <- impute(DF$Age, median)

In [ ]:
#@title
DF %>% summary()

* train / test 분리

In [ ]:
#@title
index <- createDataPartition(y = DF$Survived, p = 0.7, list = FALSE)
train <- DF[index, ]
test <- DF[-index, ]

train %>% show()
test %>% show()

* **train을 이용한 glm 모델 생성**

In [ ]:
#@title
m <- glm(Survived ~ ., data = train, family = "binomial")
summary(m)

* **유의하지 않은 설명변수 제거(step 이용)**

In [ ]:
#@title
m2 <- step(m, direction = "backward")

In [ ]:
#@title
summary(m2)

* **test를 이용한 예측**

In [ ]:
#@title
predict_value <- predict(m2, test, type = "response") %>% tibble(predict_value = .)
predict_check <- test %>% select(Survived) %>% dplyr::bind_cols(., predict_value) 
predict_check %>% show()

In [ ]:
#@title
roc <- roc(predict_check$Survived, predict_check$predict_value)
plot.roc(roc,   
        col="royalblue",  
        print.auc=TRUE, 
        max.auc.polygon=TRUE,   
        print.thres=TRUE, print.thres.pch=19, print.thres.col = "red",
        auc.polygon=TRUE, auc.polygon.col="#D1F2EB")

In [ ]:
#@title
pROC::coords(roc, "best", ret = "threshold", transpose = F)

* 생성한 모델을 이용하여 kaggle에 제출해보기 ㅎ

In [ ]:
#@title
test_kaggle <- fread("https://raw.githubusercontent.com/Jin0331/TA/master/data/titanic/test.csv", sep = ",") %>% as_tibble()
test_kaggle %>% summary()

* NA 값 추정(median)

In [ ]:
#@title
test_kaggle$Age <- impute(test_kaggle$Age, median)
test_kaggle %>% summary()

* 범주형 변수

In [ ]:
#@title
test_kaggle <- test_kaggle %>% 
 select(-Name, -Cabin, -Ticket) %>% mutate_at(c("Sex","Embarked"), factor)
summary(test_kaggle)

* 예측

In [ ]:
#@title
predict_value <- predict(m2, test_kaggle, type = "response") %>% tibble(Survived = .)
predict_cutoff <- test_kaggle %>% select(PassengerId) %>% dplyr::bind_cols(., predict_value) %>%
 mutate(Survived = ifelse(Survived >= 0.4169107, 1, 0))

In [ ]:
#@title
predict_cutoff %>% show()

In [ ]:
#@title
predict_cutoff %>% write_csv(path = "submission.csv")

In [ ]:
#@title
